# 2. PyTorch dataset and data loader

In the previous notebook, we have learned how to investigate our dataset. Next up, we'll want to create a PyTorch `Dataset` and a PyTorch `DataLoader`. These will help us get our data ready to be passed through a neural network.

## 2.1 Creating a PyTorch dataset

A [PyTorch `Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) is an abstract representation of a dataset. You can do two things with a PyTorch `Dataset`: get the *item* at a certain **index**, and get the **length** of the dataset (i.e., the number of *items*). That's all. What an *item* is, is entirely up to you, but in the case of classification, it is typically a tuple of an image and a class label.

### 2.1.1 Built-in datasets

Torchvision already includes [many datasets](https://pytorch.org/vision/stable/datasets.html) to play around with. Let's take a look at the [MNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST) dataset.

In [1]:
from torchvision.datasets import MNIST

mnist_ds = MNIST(root='../data', download=True)

We can ask an item at a certain index:

In [2]:
mnist_ds[0]

(<PIL.Image.Image image mode=L size=28x28>, 5)

This gives us two things: an image and an integer label.

In [3]:
img, label = mnist_ds[10]
img

We can also ask the length of the dataset.

In [4]:
len(mnist_ds)

60000

### 2.1.2 A first attempt to a custom dataset

To implement your own dataset, you need create a class that inherits from [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) and implements two methods: `__getitem__()` and `__len__()`.

For our Pokémon use case:

- `__getitem__()` should return the image at the given index, along with the name of that Pokémon;
- `__len__()` should return the number of images.

In the previous notebook, we saw how we can represent our dataset as a Pandas `DataFrame`. We can use this representation to implement `__getitem__()` and `__len__()` without much new code:

In [5]:
import warnings

# Ignore 'DataFrame.swapaxes' is deprecated warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from pathlib import Path
from torch.utils.data import Dataset
from PIL import Image


class PokemonDataset(Dataset):
    def __init__(self, data_path, transform=None):
        data_path = Path(data_path)
        df = pd.DataFrame([
            {
                'image': str(img_path),
                'label': img_path.parent.name
            }
            for img_path in data_path.glob('data/*/*')
        ])

        # Store attributes
        self.data_path = data_path
        self.transform = transform
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df['image'][idx]
        label = self.df['label'][idx]

        img = Image.open(img_path)

        if self.transform is not None:
            img = self.transform(img)

        return (img, label)

We can play around with this dataset, just like with the MNIST dataset.

In [6]:
ds = PokemonDataset('../data/PokemonGen1')
ds[0]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=200x279>, 'Exeggcute')

While this is already a perfectly valid PyTorch `Dataset`, there are two issues:

1. Our labels are strings;
2. We don't make a distinction between train, validation and test data.

> 🤔 **Why is it a problem that our labels are strings?**
>
> When training a network, we typically iterate over *batches* of data. These batches are represented as PyTorch tensors. And it is not possible (and not necessary, really) to create a tensor of strings.

We'll solve both issues in the following sections.

### 2.1.3 Replacing string labels with integer labels

First, we'll replace our string labels by integer labels. For this, we'll create a dictionary that maps each string label to a unique integer.

In [7]:
data_path = Path('../data/PokemonGen1')
df = pd.DataFrame([
    {
        'image': str(img_path),
        'label': img_path.parent.name
    }
    for img_path in data_path.glob('data/*/*')
])

To get a list of unique labels, we can call [`unique()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html) on the `label` column:

In [8]:
df['label'].unique()

array(['Exeggcute', 'Growlithe', 'Lickitung', 'Rhydon', 'Machop',
       'Haunter', 'Pidgey', 'Weepinbell', 'Spearow', 'Vileplume',
       'Venonat', 'Kabuto', 'Poliwrath', 'MrMime', 'Omanyte', 'Clefairy',
       'Tangela', 'Aerodactyl', 'Parasect', 'Wartortle', 'Graveler',
       'Dratini', 'Kingler', 'Kakuna', 'Gengar', 'Doduo', 'Dragonair',
       'Porygon', 'Sandshrew', 'Weedle', 'Pidgeotto', 'Poliwag',
       'Nidorino', 'Lapras', 'Nidorina', 'Charmander', 'Metapod', 'Eevee',
       'Gyarados', 'Pikachu', 'Dugtrio', 'Meowth', 'Squirtle', 'Golduck',
       'Muk', 'Grimer', 'Mankey', 'Onix', 'Tentacool', 'Machoke',
       'Cubone', 'Butterfree', 'Ninetales', 'Ekans', 'Cloyster',
       'Dragonite', 'Slowpoke', 'Raticate', 'Bulbasaur', 'Jigglypuff',
       'Seadra', 'Pidgeot', 'Voltorb', 'Rapidash', 'Drowzee', 'Ponyta',
       'Wigglytuff', 'Dewgong', 'Clefable', 'Electrode', 'Scyther',
       'Articuno', 'Charizard', 'Dodrio', 'Magmar', 'Nidoking', 'Marowak',
       'Psyduck', 'Star

To ensure a consistent mapping across different systems, we can sort these labels with [the built-in Python function `sorted()`](https://docs.python.org/3/library/functions.html#sorted):

In [9]:
sorted(df['label'].unique())

['Abra',
 'Aerodactyl',
 'Alakazam',
 'Arbok',
 'Arcanine',
 'Articuno',
 'Beedrill',
 'Bellsprout',
 'Blastoise',
 'Bulbasaur',
 'Butterfree',
 'Caterpie',
 'Chansey',
 'Charizard',
 'Charmander',
 'Charmeleon',
 'Clefable',
 'Clefairy',
 'Cloyster',
 'Cubone',
 'Dewgong',
 'Diglett',
 'Ditto',
 'Dodrio',
 'Doduo',
 'Dragonair',
 'Dragonite',
 'Dratini',
 'Drowzee',
 'Dugtrio',
 'Eevee',
 'Ekans',
 'Electabuzz',
 'Electrode',
 'Exeggcute',
 'Exeggutor',
 'Farfetchd',
 'Fearow',
 'Flareon',
 'Gastly',
 'Gengar',
 'Geodude',
 'Gloom',
 'Golbat',
 'Goldeen',
 'Golduck',
 'Golem',
 'Graveler',
 'Grimer',
 'Growlithe',
 'Gyarados',
 'Haunter',
 'Hitmonchan',
 'Hitmonlee',
 'Horsea',
 'Hypno',
 'Ivysaur',
 'Jigglypuff',
 'Jolteon',
 'Jynx',
 'Kabuto',
 'Kabutops',
 'Kadabra',
 'Kakuna',
 'Kangaskhan',
 'Kingler',
 'Koffing',
 'Krabby',
 'Lapras',
 'Lickitung',
 'Machamp',
 'Machoke',
 'Machop',
 'Magikarp',
 'Magmar',
 'Magnemite',
 'Magneton',
 'Mankey',
 'Marowak',
 'Meowth',
 'Metapod',


Now, with [the built-in Python function `enumerate()`](https://docs.python.org/3/library/functions.html#enumerate), we can get an iterable that yields another integer value for each of our labels.

In [10]:
for i, label in enumerate(sorted(df['label'].unique())):
    print(i, label)

0 Abra
1 Aerodactyl
2 Alakazam
3 Arbok
4 Arcanine
5 Articuno
6 Beedrill
7 Bellsprout
8 Blastoise
9 Bulbasaur
10 Butterfree
11 Caterpie
12 Chansey
13 Charizard
14 Charmander
15 Charmeleon
16 Clefable
17 Clefairy
18 Cloyster
19 Cubone
20 Dewgong
21 Diglett
22 Ditto
23 Dodrio
24 Doduo
25 Dragonair
26 Dragonite
27 Dratini
28 Drowzee
29 Dugtrio
30 Eevee
31 Ekans
32 Electabuzz
33 Electrode
34 Exeggcute
35 Exeggutor
36 Farfetchd
37 Fearow
38 Flareon
39 Gastly
40 Gengar
41 Geodude
42 Gloom
43 Golbat
44 Goldeen
45 Golduck
46 Golem
47 Graveler
48 Grimer
49 Growlithe
50 Gyarados
51 Haunter
52 Hitmonchan
53 Hitmonlee
54 Horsea
55 Hypno
56 Ivysaur
57 Jigglypuff
58 Jolteon
59 Jynx
60 Kabuto
61 Kabutops
62 Kadabra
63 Kakuna
64 Kangaskhan
65 Kingler
66 Koffing
67 Krabby
68 Lapras
69 Lickitung
70 Machamp
71 Machoke
72 Machop
73 Magikarp
74 Magmar
75 Magnemite
76 Magneton
77 Mankey
78 Marowak
79 Meowth
80 Metapod
81 Mew
82 Mewtwo
83 Moltres
84 MrMime
85 Muk
86 Nidoking
87 Nidoqueen
88 Nidorina
89 Nidori

So, we can create our label-to-integer dictionary with the following `for` loop:

In [11]:
label_to_int = {}

for i, label in enumerate(sorted(df['label'].unique())):
    label_to_int[label] = i

label_to_int

{'Abra': 0,
 'Aerodactyl': 1,
 'Alakazam': 2,
 'Arbok': 3,
 'Arcanine': 4,
 'Articuno': 5,
 'Beedrill': 6,
 'Bellsprout': 7,
 'Blastoise': 8,
 'Bulbasaur': 9,
 'Butterfree': 10,
 'Caterpie': 11,
 'Chansey': 12,
 'Charizard': 13,
 'Charmander': 14,
 'Charmeleon': 15,
 'Clefable': 16,
 'Clefairy': 17,
 'Cloyster': 18,
 'Cubone': 19,
 'Dewgong': 20,
 'Diglett': 21,
 'Ditto': 22,
 'Dodrio': 23,
 'Doduo': 24,
 'Dragonair': 25,
 'Dragonite': 26,
 'Dratini': 27,
 'Drowzee': 28,
 'Dugtrio': 29,
 'Eevee': 30,
 'Ekans': 31,
 'Electabuzz': 32,
 'Electrode': 33,
 'Exeggcute': 34,
 'Exeggutor': 35,
 'Farfetchd': 36,
 'Fearow': 37,
 'Flareon': 38,
 'Gastly': 39,
 'Gengar': 40,
 'Geodude': 41,
 'Gloom': 42,
 'Golbat': 43,
 'Goldeen': 44,
 'Golduck': 45,
 'Golem': 46,
 'Graveler': 47,
 'Grimer': 48,
 'Growlithe': 49,
 'Gyarados': 50,
 'Haunter': 51,
 'Hitmonchan': 52,
 'Hitmonlee': 53,
 'Horsea': 54,
 'Hypno': 55,
 'Ivysaur': 56,
 'Jigglypuff': 57,
 'Jolteon': 58,
 'Jynx': 59,
 'Kabuto': 60,
 'Kabutop

Or, we can use a [dictionary comprehension](https://www.geeksforgeeks.org/python-dictionary-comprehension/):

In [12]:
label_to_int = {label: i for i, label in enumerate(sorted(df['label'].unique()))}
label_to_int

{'Abra': 0,
 'Aerodactyl': 1,
 'Alakazam': 2,
 'Arbok': 3,
 'Arcanine': 4,
 'Articuno': 5,
 'Beedrill': 6,
 'Bellsprout': 7,
 'Blastoise': 8,
 'Bulbasaur': 9,
 'Butterfree': 10,
 'Caterpie': 11,
 'Chansey': 12,
 'Charizard': 13,
 'Charmander': 14,
 'Charmeleon': 15,
 'Clefable': 16,
 'Clefairy': 17,
 'Cloyster': 18,
 'Cubone': 19,
 'Dewgong': 20,
 'Diglett': 21,
 'Ditto': 22,
 'Dodrio': 23,
 'Doduo': 24,
 'Dragonair': 25,
 'Dragonite': 26,
 'Dratini': 27,
 'Drowzee': 28,
 'Dugtrio': 29,
 'Eevee': 30,
 'Ekans': 31,
 'Electabuzz': 32,
 'Electrode': 33,
 'Exeggcute': 34,
 'Exeggutor': 35,
 'Farfetchd': 36,
 'Fearow': 37,
 'Flareon': 38,
 'Gastly': 39,
 'Gengar': 40,
 'Geodude': 41,
 'Gloom': 42,
 'Golbat': 43,
 'Goldeen': 44,
 'Golduck': 45,
 'Golem': 46,
 'Graveler': 47,
 'Grimer': 48,
 'Growlithe': 49,
 'Gyarados': 50,
 'Haunter': 51,
 'Hitmonchan': 52,
 'Hitmonlee': 53,
 'Horsea': 54,
 'Hypno': 55,
 'Ivysaur': 56,
 'Jigglypuff': 57,
 'Jolteon': 58,
 'Jynx': 59,
 'Kabuto': 60,
 'Kabutop

We can plug this into our PokemonDataset:

In [13]:
class PokemonDataset(Dataset):
    def __init__(self, data_path, transform=None):
        data_path = Path(data_path)
        df = pd.DataFrame([
            {
                'image': str(img_path),
                'label': img_path.parent.name
            }
            for img_path in Path(data_path).glob('data/*/*')
        ])

        # Create mapping from label to integer
        label_to_int = {
            label: i
            for i, label in enumerate(sorted(df['label'].unique()))
        }

        # Store attributes
        self.data_path = data_path
        self.label_to_int = label_to_int
        self.transform = transform
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df['image'][idx]
        label = self.df['label'][idx]
        int_label = self.label_to_int[label]

        img = Image.open(img_path)

        if self.transform is not None:
            img = self.transform(img)

        return (img, int_label)

Let's try out our new dataset.

In [14]:
ds = PokemonDataset('../data/PokemonGen1/')
ds[0]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=200x279>, 34)

### 2.1.4 Splitting the data into a train, validation and test split

Training a neural network and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would achieve a perfect score but would fail to predict anything useful on data that was not used during training. This situation is called **overfitting**. To get a better sense of model performance on *unseen* data, it is customary to randomly split the dataset into three disjoint subsets.

1. **Training set**: You train the model with the training set.
2. **Validation set**: Every X training iterations, you evaluate the model performance on the validation set. It is common to only keep the model that obtained the best validation score during training. When training multiple models with different configurations (or *hyperparameters*), you should use the evaluation on the validation set to decide which model to keep.
3. **Test set**: *Once you have decided a model based on validation perfomance*, you evaluate on the test set to get an estimate of the model's ability to generalize to unseen data. In many machine learning competitions, to avoid cheating, the test set is either unlabeled or unavailable to the participants.

As you see, the validation set is used to compare different model configurations, or *hyperparameters*. This could simply be the number of iterations used to train the model, but this also includes the choice of data augmentations, optimizer, learning rate, model architecture (number of layers, number of neurons in a layer, kind of layer,...) etc.

To split our dataset into these subsets, we can use [`sklearn.model_selection.train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split).

In [15]:
from sklearn.model_selection import train_test_split

df_trainval, df_test = train_test_split(df, train_size=0.8, random_state=42)

df_train, df_val = train_test_split(df_trainval, train_size=0.8, random_state=42)

We can update our `PokemonDataset` so that, depending on a given constructor argument, the correct subset is used.

In [16]:
class PokemonDataset(Dataset):
    def __init__(self, data_path, subset, transform=None):
        data_path = Path(data_path)
        df = pd.DataFrame([
            {
                'image': str(img_path),
                'label': img_path.parent.name
            }
            for img_path in data_path.glob('data/*/*')
        ])

        # Create mapping from label to integer
        label_to_int = {
            label: i
            for i, label in enumerate(sorted(df['label'].unique()))
        }

        # Split into train, test, val
        df_trainval, df_test = train_test_split(df, train_size=0.8, random_state=42)
        df_train, df_val = train_test_split(df_trainval, train_size=0.8, random_state=42)

        # Store attributes
        self.data_path = data_path
        self.label_to_int = label_to_int
        self.transform = transform
        self.subset = subset

        if subset == 'train':
            self.df = df_train.reset_index()
        elif subset == 'val':
            self.df = df_val.reset_index()
        elif subset == 'test':
            self.df = df_test.reset_index()
        else:
            raise ValueError(f'Unknown subset "{subset}"')

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df['image'][idx]
        label = self.df['label'][idx]
        int_label = self.label_to_int[label]

        img = Image.open(img_path)

        if self.transform is not None:
            img = self.transform(img)

        return (img, int_label)

Let's try it out...

In [17]:
ds_train = PokemonDataset(data_path='../data/PokemonGen1', subset='train')
ds_train[0]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=453x398>, 77)

### 2.1.5 Cross-validation

Choosing the model that achieved the best performance on the validation set (instead of the training set) avoids selecting a model that has overfit. However, when comparing many models based on validation score, we might select a model that only worked best for the particular random choice of training and validation data. To make a better-informed decision, we can use **cross-validation** techniques.

A basic cross-validation technique is **$k$-fold cross-validation**. Here, the data set is first split up into only two subsets: a *training* set and a *test* set. The training set is partitioned into **$k$ equally-sized folds**. When training a model, we choose one fold for *validation* and the other $k - 1$ folds for *training*. By training a certain model configuration with all $k$ different choices for training and validation and averaging the validation scores, we can effectively select the best model configuration in a more reliable and robust manner. The figure below illustrates the process of $k$-fold cross-validation ([source](https://scikit-learn.org/stable/modules/cross_validation.html)).

<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" style="max-width: 500px; margin: auto; padding: 2em;"/>

To add support for $k$-fold cross validation to our `PokemonDataset`, we'll first use `train_test_split()` to split our dataset into train+val and test set. Then, with [`numpy.array_split()`](https://numpy.org/doc/stable/reference/generated/numpy.array_split.html), we can split the train+val set into $k$ folds.

In [18]:
import numpy as np

# Define k (the number of folds)
k = 5

# Split dataset into train+val and test
df_trainval, df_test = train_test_split(df, train_size=0.8, random_state=42)

# Split the train+val into k folds
folds = np.array_split(df_trainval, k)

/home/fdf/projects/labo_bld_interp/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Now, we can choose which fold to use for validation. The other folds should be concatenated to create the training set.

In [19]:
val_fold = 0

df_val = folds[val_fold]
train_folds = [fold for i, fold in enumerate(folds)
               if i != val_fold]
df_train = pd.concat(train_folds)

Plugging this into our `PokemonDataset`:

In [20]:
class PokemonDataset(Dataset):
    def __init__(self, data_path, subset, k=5, val_fold=0, transform=None):
        data_path = Path(data_path)
        df = pd.DataFrame([
            {
                'image': str(img_path),
                'label': img_path.parent.name
            }
            for img_path in data_path.glob('data/*/*')
        ])

        # Create mapping from label to integer
        label_to_int = {
            label: i
            for i, label in enumerate(sorted(df['label'].unique()))
        }

        # Split into train, test, val
        df_trainval, df_test = train_test_split(df, train_size=0.8, random_state=42)
        folds = np.array_split(df_trainval, k)
        df_val = folds[val_fold]
        train_folds = [fold for i, fold in enumerate(folds)
                       if i != val_fold]
        df_train = pd.concat(train_folds)

        # Store attributes
        self.data_path = data_path
        self.label_to_int = label_to_int
        self.transform = transform

        if subset == 'train':
            self.df = df_train.reset_index()
        elif subset == 'val':
            self.df = df_val.reset_index()
        elif subset == 'test':
            self.df = df_test.reset_index()
        else:
            raise ValueError(f'Unknown subset "{subset}"')

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df['image'][idx]
        label = self.df['label'][idx]
        int_label = self.label_to_int[label]

        img = Image.open(img_path)

        if self.transform is not None:
            img = self.transform(img)

        return (img, int_label)

Now we can create a training set, validation set and test set:

In [21]:
import torch
from torchvision.transforms import v2

# Create transforms
train_tfms = v2.Compose([
    v2.ToImage(),
    v2.RandomHorizontalFlip(),
    v2.RandomRotation(30),
    v2.RandomResizedCrop(224, antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_tfms = v2.Compose([
    v2.ToImage(),
    v2.Resize(224, antialias=True),
    v2.CenterCrop(224),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets
ds_train = PokemonDataset('../data/PokemonGen1', 'train', transform=train_tfms)
ds_val = PokemonDataset('../data/PokemonGen1', 'val', transform=val_tfms)
ds_test = PokemonDataset('../data/PokemonGen1', 'test', transform=val_tfms)

/home/fdf/projects/labo_bld_interp/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/fdf/projects/labo_bld_interp/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


> ⚠️ **Don't use random transforms for validation or test set!**
>
> To avoid making the validation and test evaluations irreproducible, you should not use randomness in the transforms that you'll use for the validation and test set.

Hooray! 🙌 We now have a full-fledged PyTorch `Dataset` with support for $k$-fold cross-validation! 🎉

## 2.2 PyTorch data loader

A PyTorch `Dataset` allows us to easily get images (and labels) from our dataset. When training a neural network, however, we can save training time by training with **batches** of data, instead of passing the images through the network one-by-one.

Batching your data is the task of the [PyTorch `DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader). Given a PyTorch `Dataset` that returns a tuple with an image tensor and an integer label, it is very simple to create a `DataLoader`:

In [22]:
from torch.utils.data import DataLoader

# Create trainin data loader
dl_train = DataLoader(ds_train, batch_size=10)

A `DataLoader` is an iterable, where each iteration item is a batch of data samples. As with any iterable in Python, you can iterate over a `DataLoader` using a `for` loop. Let's inspect what's inside an iteration item.

In [23]:
for x in dl_train:
    break

x

[tensor([[[[ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
           [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
           [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
           ...,
           [ 2.2489,  2.2489,  2.2489,  ...,  0.5536,  0.5364,  0.5364],
           [ 2.2489,  2.2489,  2.2489,  ...,  0.5536,  0.5536,  0.5536],
           [ 2.2489,  2.2489,  2.2489,  ...,  0.5536,  0.5536,  0.5536]],
 
          [[ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
           [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
           [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
           ...,
           [ 2.4286,  2.4286,  2.4286,  ...,  0.7654,  0.7304,  0.7304],
           [ 2.4286,  2.4286,  2.4286,  ...,  0.7654,  0.7479,  0.7479],
           [ 2.4286,  2.4286,  2.4286,  ...,  0.7829,  0.7654,  0.7479]],
 
          [[ 2.6400,  2.6400,  2.6400,  ...,  2.6400,  2.6400,  2.6400],
           [ 

Our `DataLoader` composes the first batch with the samples at index $0,\ldots, 9$, the second batch $10,\ldots,19$ and so on. To compose the batches with random samples, you can pass `shuffle=True` to the `DataLoader` constructor.

> ⚠️ **Only shuffle the training set!**
>
> It is considered bad practice to shuffle any of your evaluation sets, as this might make your evaluations irreproducible.

In [24]:
dl_train = DataLoader(ds_train, batch_size=10, shuffle=True)

When taking a look at the implementation of `__getitem__()` in our `PokemonDataset`, you'll see that it involves loading an image from disk with `read_image()` and applying image transforms. Both of these steps may block the computing process for a while. If we'd sequentially run `__getitem__()` for the 10 indices in our batch, it might take some time before the batch is created. Setting the argument `num_workers` as a positive integer will turn on **multi-process data loading** with the specified number of loader worker processes.

In [25]:
dl_train = DataLoader(ds_train, batch_size=10, shuffle=True, num_workers=10)

In summary, the following code creates our train, validation and test datasets and data loaders.

In [26]:
# Create data loaders
dl_train = DataLoader(ds_train, batch_size=10, shuffle=True)
dl_val = DataLoader(ds_val, batch_size=10, shuffle=False)
dl_test = DataLoader(ds_test, batch_size=10, shuffle=False)

And with this, we have covered everything on the data side! 💪 Our data is now ready to be passed into a neural network!